In [23]:
from lxml import etree
import re, time
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [4]:
class Qiangpiao(object):
    def __init__(self):
        self.driver_path = '/Users/myki/Documents/Python/Spider/chromedriver'
        self.driver = webdriver.Chrome(self.driver_path)
        #登录的页面
        self.login_url = 'https://kyfw.12306.cn/otn/resources/login.html'
        #登录后的页面
        self.initmy_url = 'https://kyfw.12306.cn/otn/view/index.html'
        #查询票的页面
        self.search_url = 'https://kyfw.12306.cn/otn/leftTicket/init?linktypeid=dc'
        self.passenger_url = 'https://kyfw.12306.cn/otn/confirmPassenger/initDc'

    def wait_input(self):
        self.from_station = input('起始站:')
        self.to_station = input('目的地:')
        self.depart_time = input('出发时间:')
        self.passengers = input('乘客姓名（若有多个，则用, 隔开）').split(',')
        self.trains = input('车次（若有多个，用, 隔开）').split(',')
        
        
    def login(self):
        self.driver.get(self.login_url)
        print('已进入登录页面.')
        print('. '*30 + '\n')
        
        loginBtn = self.driver.find_element_by_xpath("//li[@class='login-hd-account']")
        loginBtn.click()
        print("点击切换完成，开始自动填写用户名和密码...")
        print('. '*30 + '\n')
        
        loginName = self.driver.find_element_by_xpath("//input[@id='J-userName']")
        loginName.send_keys('njlgdxedu@163.com')
        password = self.driver.find_element_by_xpath("//input[@type='password']")
        password.send_keys('173Ling172')
        print("用户名和密码自动填写完毕！")
        
        WebDriverWait(self.driver, 1000).until(
            EC.url_to_be(self.initmy_url)
        )
        print('登录成功！^O^!')
        
    
    def _order_ticket(self):
        #跳转至查询余票的界面
        self.driver.get(self.search_url)
        print("已跳转至查询余票的界面！")
        
        input_from_station = self.driver.find_element_by_id("fromStationText")
        input_from_station.send_keys(self.from_station)
        #等待出发地是否输入正确
        WebDriverWait(self.driver, 1000).until(
            EC.text_to_be_present_in_element_value((By.ID, "fromStationText"), self.from_station)
        )
        print("出发地输入正确")
        
        
        input_to_station = self.driver.find_element_by_id("toStationText")
        input_to_station.send_keys(self.to_station)
        #等待目的地是否输入正确
        WebDriverWait(self.driver, 1000).until(
            EC.text_to_be_present_in_element_value((By.ID, 'toStationText'), self.to_station)
        )
        print('目的地输入正确')
        
        input_train_date = self.driver.find_element_by_id("train_date")
        input_train_date.send_keys(self.depart_time)
        #等待出发日期是否输入正确
        WebDriverWait(self.driver, 1000).until(
            EC.text_to_be_present_in_element_value((By.ID, 'train_date'), self.depart_time)
        )
        print('出发日期输入正确')
        
        #等待查询按钮是否可点击
        WebDriverWait(self.driver, 1000).until(
            EC.element_to_be_clickable((By.ID, 'query_ticket'))
        )
        print('查询按钮可以开始点击')
        
        #如果可点击，则点击该查询按钮
        searchBtn = self.driver.find_element_by_id('query_ticket')
        searchBtn.click()
        print('已经开始查询...')
        print('. '*30 + '\n')
        
        #在点击查询之后， 等待车次信息是否显示出来
        WebDriverWait(self.driver, 1000).until(
            EC.presence_of_element_located((By.XPATH, ".//tbody[@id='queryLeftTable']/tr"))
        )
        
        #找到所有没有datatrain属性的tr标签
        tr_list = self.driver.find_elements_by_xpath(".//tbody[@id='queryLeftTable']/tr[not(@datatran)]")
        
        #遍历所有满足条件的tr标签
        for tr in tr_list:
            train_number = tr.find_element_by_class_name("number").text
            if train_number in self.trains:
                left_ticket = tr.find_element_by_xpath(".//td[4]").text
                if left_ticket == '有' or left_ticket.isdigit:
                    print(train_number + ' 有票！！happy^_^')
                    orderBtn = tr.find_element_by_class_name("btn72")
                    orderBtn.click()
                    
                    #等待是否跳转至订票信息确认页面
                    WebDriverWait(self.driver, 1000).until(
                        EC.url_to_be(self.passenger_url)
                    )
                    print('已经跳转至订票信息页面， 请确认订单信息.')
                    break
                else:
                    print(train_number + ' 无票~sad~~~~>_<')
        
        WebDriverWait(self.driver, 1000).until(
            EC.element_to_be_clickable((By.ID, 'normalPassenger_4'))
        )
        #confirmBtn = self.driver.find_elements_by_xpath("//input[@id='normalPassenger_0']")
        #confirmBtn.checked()
        
        submitBtn = self.driver.find_element_by_id('submitOrder_id')
        submitBtn.click()
        print('已经提交订单，正在处理中...')
        
        WebDriverWait(self.driver, 1000).until(
            EC.element_to_be_clickable((By.ID, 'normalPassenger_5'))
        )
        confirm2Btn = self.driver.find_element_by_id('qr_submit_id')
        confirm2Btn.click()
        print('订单完成^O^')
        
    
    def run(self):
        self.wait_input()
        self.login()
        self._order_ticket()
        
if __name__ == '__main__':
    spider = Qiangpiao()
    spider.run()

起始站:南京 
目的地:北京 
出发时间:2020-02-02
乘客姓名（若有多个，则用, 隔开）CHEN JINYUAN
车次（若有多个，用, 隔开）D401
已进入登录页面.
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

点击切换完成，开始自动填写用户名和密码...
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

用户名和密码自动填写完毕！


NoSuchWindowException: Message: no such window: window was already closed
  (Session info: chrome=79.0.3945.130)


In [82]:
#!/usr/bin/env python
# author: Myki time: 22/1/20
import csv

from lxml import etree
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


class digData(object):
    def __init__(self):
        self.driver_path = '/Users/myki/Documents/Python/Spider/chromedriver'
        self.driver = webdriver.Chrome(self.driver_path)

        self.login_url = 'https://www.thepartshome.com.au/customer/account/login/'

        self.initmy_url = 'https://www.thepartshome.com.au/customer/account/'

        self.products = []

    def login(self):
        self.driver.get(self.login_url)
        print('Have entered the interface of log in.')

        WebDriverWait(self.driver, 1000).until(
            EC.element_to_be_clickable((By.ID, 'send2'))
        )

        username = self.driver.find_element_by_xpath("//*[@id='email']")
        username.send_keys('chenmyki@gmail.com')
        password = self.driver.find_element_by_xpath("//*[@id='pass']")
        password.send_keys('173ling172')

        loginBtn = self.driver.find_element_by_xpath("//*[@id='send2']")
        loginBtn.click()

        WebDriverWait(self.driver, 1000).until(
            EC.element_to_be_clickable((By.ID, 'search'))
        )
        print('Log in successfully！^O^!')

        search = self.driver.find_element_by_xpath("//*[@id='search']")
        search.send_keys('iphone 8')

        searchBtn = self.driver.find_element_by_xpath("//*[@id='search_mini_form']/div/button")
        searchBtn.click()

        WebDriverWait(self.driver, 1000).until(
            EC.element_to_be_clickable((By.ID, 'price-excluding-tax-2965'))
        )

        # prices = self.driver.find_element_by_xpath("//span[@class='price']")
        source = self.driver.page_source
        self.parse_detail_page(source)

    def parse_detail_page(self, source):
        html = etree.HTML(source)
        # names = html.xpath("//h2[@class='product-name']//text()")
        # prices = html.xpath("//span[@class='price']//text()")
        lis = html.xpath("//div[@class='category-products']/ul/li")

        for li in lis:
            name = li.xpath(".//h2[@class='product-name']/a/@title")
            price = li.xpath(".//span[@class='price']//text()")[0].strip()
            inclGST = li.xpath(".//span[@class='price-including-tax']/span[2]//text()")[0].strip()

            # print(name + price)

            product = {
                'Product Name': name,
                'Excl GST Price': price,
                'Include GST Price': inclGST
            }
            self.products.append(product)

        for product in self.products:
            print(product)

        self.saveInfo(self.products)

    def saveInfo(self, products):
        headers = [
            'Product Name', 'Excl GST Price', 'Include GST Price'
        ]
        values = self.products

        with open('digInfo.csv', 'w', encoding='utf-8', newline='') as fp:
            writer = csv.DictWriter(fp, headers)
            writer.writeheader()
            writer.writerows(values)


    def run(self):
        self.login()

if __name__ == '__main__':
    spider = digData()
    spider.run()

Have entered the interface of log in.
Log in successfully！^O^!
{'Product Name': ['iPhone 7 / 8 Ear Speaker Earpiece Replacement'], 'Price': 'AU$2.07', 'InclGST': 'AU$2.28'}
{'Product Name': ['iPhone 7 Plus / 8 Plus Ear Speaker Earpiece Replacement'], 'Price': 'AU$2.07', 'InclGST': 'AU$2.28'}
{'Product Name': ['iPhone 8 LCD Screen Replacement Assembly with Touch and Frame [Grade S+] [Black]'], 'Price': 'AU$65.00', 'InclGST': 'AU$71.50'}
{'Product Name': ['iPhone 8 LCD Screen Replacement Assembly with Touch and Frame [Grade S+] [White]'], 'Price': 'AU$65.00', 'InclGST': 'AU$71.50'}
{'Product Name': ['iPhone 8 Plus LCD Screen Replacement Assembly with Touch and Frame [Grade S+] [White]'], 'Price': 'AU$110.00', 'InclGST': 'AU$121.00'}
{'Product Name': ['iPhone 8 Plus LCD Screen Replacement Assembly with Touch and Frame [Grade S+][Black]'], 'Price': 'AU$110.00', 'InclGST': 'AU$121.00'}
{'Product Name': ['iPhone 8 Front Facing Camera with Proximity Sensor Flex Cable Ribbon'], 'Price': 'AU$26